# Libs

In [ ]:
from torchvision.datasets import CelebA
import os
import torch.nn as nn
import glob
import os
import torch
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import os
import torch
import math
# main libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from torch.nn import functional as F
from torch import nn
from torch import optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import Dataset
import math
import torchvision.transforms.functional as TF
import cv2
from keras.utils import image_dataset_from_directory
from torchvision import datasets, transforms
from torch.utils.data import random_split
from torch.nn.parameter import Parameter

from torch.nn import init
from torchvision.utils import make_grid
from collections import OrderedDict

import torch
from torch.utils.data import Dataset

from skimage import transform
import skimage.io as io
import numpy as np
from torchvision.utils import save_image

# Helper Functions

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def show_tensor_images(image_tensor, num_images=25, size=(1, 128, 128)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in a uniform grid.
    '''
    image_unflat = image_tensor.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

In [ ]:
def ploter(image, image_hat):
    """
    (H, W)
    """
    plt.figure()
    plt.subplot(1,2,1)
    #plt.imshow(image_hat, cmap='gray', vmin=-1, vmax=1)
    plt.imshow(image_hat)
    plt.tight_layout()
    plt.title("Reconstruct")

    plt.subplot(1,2,2)
    #plt.imshow(image, cmap='gray', vmin=-1, vmax=1)
    plt.imshow(image)
    plt.tight_layout()
    plt.title("Original")

    plt.show()

# Generator

In [ ]:
os.chdir("/content/drive/MyDrive/AD Last Research")

In [ ]:
from mymodels.generator import Generator_XL, Generator_S

In [ ]:
generator = Generator_XL(
    z_dim=100,
    c_dim=0,
    w_dim=0,
    img_resolution=32,
    img_channels=3,
    ngf=8,
    cond=0,
)

In [ ]:
count_parameters(generator)

In [ ]:
generator = Generator_S(features=64,
                        img_size=32,
                        latent_dim=100,
                        channels=3)
count_parameters(generator)

# Discriminator

In [ ]:
from mymodels.discriminator import Discriminator

In [ ]:
discriminator = Discriminator(features=32, channels=3, img_size=32, optimization='gan')

In [ ]:
count_parameters(discriminator)

# Dataset

In [ ]:
from dataset import customdataset

In [ ]:
img_size = 32
channels = 3
batch_size = 10

In [ ]:
# Train
train_dataset_normal = customdataset.get_nomral_dataset_train(dataset_name='cifar10',
                                                               label=0,
                                                               img_size=img_size, channels=channels,
                                                               data_path='./CIFAR10', download=True)

train_indx = SubsetRandomSampler(list(range(1000)))
train_loader_normal = DataLoader(train_dataset_normal, batch_size=batch_size, sampler=train_indx)

In [ ]:
# Test
test_dataset_normal,  test_dataset_ano = customdataset.get_dataset_test(dataset_name='cifar10',
                                                                         normal_label=0, ano_label=2,
                                                                         img_size=img_size, channels=channels,
                                                                         data_path="./CIFAR10", download=True)

test_loader_normal = DataLoader(test_dataset_normal, batch_size = test_dataset_normal.__len__())
test_loader_ano = DataLoader(test_dataset_ano, batch_size = test_dataset_ano.__len__())

# Vanilla GAN

### Hyperparameters

In [ ]:
device = 'cuda'
latent_dim = 100

### All in One

In [ ]:
# Nets
generator = Generator_XL(
    z_dim=latent_dim,
    c_dim=0,
    w_dim=0,
    img_resolution=img_size,
    img_channels=channels,
    ngf=8,
    cond=0,
).to(device)

generator = Generator_S(features=64,
                        img_size=img_size,
                        latent_dim=latent_dim,
                        channels=channels).to(device)

discriminator = Discriminator(features=16, channels=channels, img_size=img_size, optimization='gan').to(device)

In [ ]:
# Loss & Optimizers
adversarial_loss = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters())
optimizer_D = optim.Adam(discriminator.parameters())

### Re-Init

In [ ]:
# WRONG

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0.0)

In [ ]:
generator = generator.apply(weights_init_normal)
discriminator = discriminator.apply(weights_init_normal)

### Train

In [ ]:
device = 'cuda'
generator = generator.to(device)
discriminator = discriminator.to(device)

init_epoch = 0
num_epochs = 100
sample_interval = 10

In [ ]:
for epoch in range(init_epoch, num_epochs+1):
    total_G_loss = 0.0
    total_D_loss = 0.0


    torch.cuda.empty_cache()
    #del variables
    #gc.collect()
    #torch.cuda.memory_summary(device=None, abbreviated=False)

    # Load Train Data ##############################################
    for i, (real_images, _) in enumerate(train_loader_normal):

        torch.cuda.empty_cache()
        #del variables
        #gc.collect()
        #torch.cuda.memory_summary(device=None, abbreviated=False)

        
        # Preprocessing ##############################################
        valid = torch.FloatTensor(real_images.shape[0], 1).fill_(1.0).to(device)
        fake = torch.FloatTensor(real_images.shape[0], 1).fill_(0.0).to(device)
        real_images = real_images.to(device)

        # MRI
        #real_images = real_images.unsqueeze(1).cuda()
        #real_images = real_images.repeat(1,3,1,1)


        # Train Generator ########################################
        optimizer_G.zero_grad()
        z = torch.FloatTensor(np.random.normal(0, 1, (real_images.shape[0], latent_dim))).to(device)
        gen_imgs = generator(z)

        g_loss_adv = adversarial_loss(discriminator(gen_imgs), valid)
        g_loss = g_loss_adv
        g_loss.backward()
        optimizer_G.step()
        total_G_loss += g_loss.cpu().detach().numpy()



        # Train Discriminator ########################################
        optimizer_D.zero_grad()
        discriminator_opinion_real = discriminator(real_images)
        discriminator_opinion_fake = discriminator(gen_imgs.detach())
        real_loss = adversarial_loss(discriminator_opinion_real, valid)
        fake_loss = adversarial_loss(discriminator_opinion_fake, fake)
        d_loss_adv = (real_loss + fake_loss) / 2
        #d_loss_layers = discriminator_layers_loss(discriminator, vgg, real_images, lamda = 0.0)
        d_loss = d_loss_adv #+ d_loss_layers
          
        d_loss.backward()
        optimizer_D.step()
        total_D_loss += d_loss.cpu().detach().numpy()


    # Load Test Data ##############################################
    if epoch % sample_interval ==0:
        for _, (real_images, _) in enumerate(test_loader_normal):
            # Preprocessing ##############################################
            real_images = real_images.to(device)

            # MRI Image
            #real_images = real_images.unsqueeze(1).cuda()
            #real_images = real_images.repeat(1,3,1,1)

            z = torch.FloatTensor(np.random.normal(0, 1, (real_images.shape[0], latent_dim))).to(device)
            gen_imgs = generator(z)

            # Visualization ##############################################
            show_tensor_images(gen_imgs, size=(channels, img_size, img_size))
            show_tensor_images(real_images, size=(channels, img_size, img_size))

            num = math.floor(np.random.uniform(0, len(test_loader_normal)))
            ploter(real_images.detach().cpu()[num][0], gen_imgs.detach().cpu()[num][0])


        # Losses ##############################################
        print("[Epoch {}/{}] \t[D loss: {:.3f}] \t[G loss: {:.3f}]".format(epoch, num_epochs, total_D_loss, total_G_loss))
        # SAVE ##############################################
        """
        torch.save({"epoch": epoch,
        "state_dict_G": generator.state_dict(),
        "state_dict_D": discriminator.state_dict(),
        "optimizer_G": optimizer_G.state_dict(),
        "optimizer_D": optimizer_D.state_dict()
        }, root_save + f"/epoch_{epoch}.pt")
        """

In [ ]:
from sklearn.manifold import TSNE

def get_2d_tsne(x):
    tsne = TSNE(n_components=2, random_state=0,  learning_rate='auto', init='random')
    x_2d = tsne.fit_transform(x)

    return x_2d

In [ ]:
device = 'cpu'
generator = generator.to(device)
discriminator = discriminator.to(device)

for _, (real_images, _) in enumerate(test_loader_normal):
    # Preprocessing ##############################################
    real_images = real_images.to(device)

    # MRI Image
    #real_images = real_images.unsqueeze(1).cuda()
    #real_images = real_images.repeat(1,3,1,1)

    z = torch.FloatTensor(np.random.normal(0, 1, (real_images.shape[0], latent_dim))).to(device)
    gen_imgs = generator(z)

In [ ]:
d1 = discriminator.conv_layers(real_images.cpu()).flatten(1)
d2 = discriminator.conv_layers(gen_imgs.cpu()).flatten(1)

In [ ]:
d1_2d = get_2d_tsne(d1.detach().cpu())
d2_2d = get_2d_tsne(d2.detach().cpu())

In [ ]:
plt.figure()
plt.scatter(d1_2d[:,0], d1_2d[:,1], label='Real')
plt.scatter(d2_2d[:,0], d2_2d[:,1], label='Fake')
plt.legend()

# WGAN

### GP

In [ ]:
from torch.autograd import Variable
Tensor = torch.cuda.FloatTensor # Cuda
import torch.autograd as autograd

def compute_gradient_penalty(discriminator, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1))).to(real_samples.device)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = discriminator(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

### Hyperparameters

In [ ]:
device = 'cuda'
latent_dim = 100
n_critic = 5
lambda_gp =10
lr = 2e-3

### All in One

In [ ]:
# Nets
generator = Generator(
    z_dim=latent_dim,
    c_dim=0,
    w_dim=0,
    img_resolution=img_size,
    img_channels=channels,
    ngf=8,
    cond=0,
).to(device)

discriminator = Discriminator(features=32, channels=channels, img_size=img_size, optimization='wgan').to(device)

In [ ]:
# Loss & Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=lr,  betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr,  betas=(0.5, 0.999))

### Re-Init

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0.0)

In [ ]:
generator = generator.apply(weights_init_normal)
discriminator = discriminator.apply(weights_init_normal)

### Train

In [ ]:
init_epoch = 0
num_epochs = 100
sample_interval = 10

In [ ]:
for epoch in range(init_epoch, num_epochs+1):
    total_G_loss = 0.0
    total_D_loss = 0.0

    # Load Train Data ##############################################
    for i, (real_images, _) in enumerate(train_loader_normal):

        # Preprocessing ##############################################
        real_images = real_images.to(device)
        # MRI
        #real_images = real_images.unsqueeze(1).cuda()
        #real_images = real_images.repeat(1,3,1,1)


        # Train Discriminator ########################################
        z = torch.FloatTensor(np.random.normal(0, 1, (real_images.shape[0], latent_dim))).to(device)
        gen_imgs = generator(z)
        optimizer_D.zero_grad()
        real_validity = discriminator(real_images)
        fake_validity = discriminator(gen_imgs.detach())
        gradient_penalty = compute_gradient_penalty(discriminator, real_images, gen_imgs.detach())
        d_loss_adv = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty


        #d_loss_layers = discriminator_layers_loss(discriminator, vgg, real_images, lamda = 0.01)


        d_loss = d_loss_adv #+ d_loss_layers
        d_loss.backward()
        optimizer_D.step()
        total_D_loss += d_loss.cpu().detach().numpy()



        # Train Generator ########################################
        if i % n_critic == 0:
            z = torch.FloatTensor(np.random.normal(0, 1, (real_images.shape[0], latent_dim))).to(device)
            gen_imgs = generator(z)

            optimizer_G.zero_grad()
            fake_validity = discriminator(gen_imgs)
            g_loss_adv = -torch.mean(fake_validity)

            g_loss = g_loss_adv
            g_loss.backward()
            optimizer_G.step()
            total_G_loss += g_loss.cpu().detach().numpy()


    # Load Test Data ##############################################
    if epoch % sample_interval ==0:
        for _, (real_images, _) in enumerate(test_loader_normal):
            # Preprocessing ##############################################
            real_images = real_images.to(device)
            # MRI Image
            #real_images = real_images.unsqueeze(1).cuda()
            #real_images = real_images.repeat(1,3,1,1)

            z = torch.FloatTensor(np.random.normal(0, 1, (real_images.shape[0], latent_dim))).to(device)
            gen_imgs = generator(z)

            # Visualization ##############################################
            show_tensor_images(gen_imgs, size=(channels, img_size, img_size))
            show_tensor_images(real_images, size=(channels, img_size, img_size))
            num = math.floor(np.random.uniform(0, len(test_loader_normal)))
            ploter(real_images.detach().cpu()[num][0], gen_imgs.detach().cpu()[num][0])


        # Losses ##############################################
        print("[Epoch {}/{}] \t[D loss: {:.3f}] \t[G loss: {:.3f}]".format(epoch, num_epochs, total_D_loss, total_G_loss))
        # SAVE ##############################################
        """
        torch.save({"epoch": epoch,
        "state_dict_G": generator.state_dict(),
        "state_dict_D": discriminator.state_dict(),
        "optimizer_G": optimizer_G.state_dict(),
        "optimizer_D": optimizer_D.state_dict()
        }, root_save + f"/epoch_{epoch}.pt")
        """